## Ablation Study for multi-modal integration

#### Step 1: Setup

First, ensure you have your multi-modal embeddings ready, which consist of both text and image embeddings. You will also need a dataset for evaluation and a method for calculating retrieval metrics.

#### Step 2: Define Functions for Ablation
You will create functions to perform the ablation by removing images from the embeddings and then evaluate retrieval performance.

In [ ]:
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity

def ablate_images(multi_modal_embeddings, ablation_fraction):
    """
    Remove a fraction of image embeddings from the multi-modal embeddings.
    
    Args:
        multi_modal_embeddings (torch.Tensor): The combined multi-modal embeddings.
        ablation_fraction (float): Fraction of images to remove (0.0 to 1.0).
        
    Returns:
        torch.Tensor: The modified multi-modal embeddings with images removed.
    """
    # Assume multi_modal_embeddings is a tensor of shape (N, D) where N is the number of samples
    # and D is the dimension of the embedding.
    
    num_samples = multi_modal_embeddings.shape[0]
    num_images_to_remove = int(num_samples * ablation_fraction)
    
    # Randomly select indices to remove
    indices_to_remove = np.random.choice(num_samples, num_images_to_remove, replace=False)
    
    # Create a mask that keeps only the non-removed indices
    mask = np.ones(num_samples, dtype=bool)
    mask[indices_to_remove] = False
    
    # Return the modified embeddings
    return multi_modal_embeddings[mask]

def evaluate_retrieval(embeddings, query_embeddings):
    """
    Evaluate retrieval performance using cosine similarity.
    
    Args:
        embeddings (torch.Tensor): The embeddings to retrieve from.
        query_embeddings (torch.Tensor): The query embeddings.
        
    Returns:
        float: Mean Average Precision or other relevant metric.
    """
    # Compute cosine similarity between query and embeddings
    similarities = cosine_similarity(query_embeddings.cpu(), embeddings.cpu())
    
    # Here you would implement your retrieval metric calculation,
    # e.g., Mean Average Precision (MAP), Recall@K, etc.
    
    return np.mean(similarities)  # Placeholder for actual metric calculation


#### Step 3: Run the Ablation Study

Now, you can run your ablation study by varying the fraction of images removed and evaluating the impact on retrieval metrics.

In [ ]:
# Assume `multi_modal_embeddings` and `text_only_embeddings` are precomputed tensors
# Assume `query_embeddings` are also prepared for evaluation

ablation_fractions = [0.0, 0.5, 1.0]  # No removal, 50% removal, 100% removal
results = {}

for fraction in ablation_fractions:
    print(f"Evaluating with {fraction*100}% image removal...")
    
    # Perform ablation
    ablated_embeddings = ablate_images(multi_modal_embeddings, fraction)
    
    # Evaluate retrieval performance
    retrieval_metric = evaluate_retrieval(ablated_embeddings, query_embeddings)
    
    results[fraction] = retrieval_metric
    print(f"Retrieval Metric for {fraction*100}% image removal: {retrieval_metric:.4f}")

# Print final results
print("Ablation Results:")
for fraction, metric in results.items():
    print(f"Fraction Removed: {fraction*100}%, Retrieval Metric: {metric:.4f}")
